In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import time 
import math


In [2]:
import os

directory = 'casts'  
all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
df_list = [pd.read_csv(file) for file in all_files]
casts_df = pd.concat(df_list, ignore_index=True)

In [3]:
casts_df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
casts_df.to_csv('casts_df.csv', index=False)

In [5]:
import pandas as pd
import os

directory = 'details'  # CSV 파일이 위치한 디렉토리 경로
all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

df_list = [pd.read_csv(file) for file in all_files]
details_df = pd.concat(df_list, ignore_index=True)

/var/folders/dr/lcj3g6gx3ps0px9m71zvyj680000gn/T/ipykernel_3259/2262533351.py:7: DtypeWarning: Columns (1,23,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in all_files]


In [6]:
details_df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
details_df.to_csv('details_df.csv', index=False)

In [8]:
final_tmdb = pd.merge(details_df, casts_df, on='id', how='inner')

In [9]:
final_tmdb.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'success', 'status_code',
       'status_message', 'casts', 'director', 'director_of_photography',
       'producer', 'writer', 'music_composer'],
      dtype='object')

In [10]:
filtered_df=final_tmdb.dropna(subset='overview')

In [11]:
filtered_df2=filtered_df[filtered_df['status'].str.lower() == 'released']

In [12]:
final_filterd_tmdb = filtered_df2[['id', 'title', 'budget', 'revenue', 'genres','imdb_id',
       'overview', 'tagline', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime',
       'vote_average', 'vote_count', 'casts', 'director', 'director_of_photography',
       'producer', 'writer', 'music_composer']]

In [13]:
final_filterd_tmdb.to_csv('final_filterd_tmdb.csv', index=False)

In [14]:
# director_stats = final_filterd_tmdb.explode('director').groupby('director').agg({
#     'title': 'count', 
#     'vote_average': 'mean',
#     'vote_count': 'sum'  
# }).rename(columns={
#     'title': 'movie_count', 
#     'vote_average': 'avg_rating',
#     'vote_count': 'total_votes'
# }).reset_index()

# actor_stats = final_filterd_tmdb.explode('casts').groupby('casts').agg({
#     'title': 'count', 
#     'vote_average': 'mean',
#     'vote_count': 'sum'  
# }).rename(columns={
#     'title': 'movie_count', 
#     'vote_average': 'avg_rating',
#     'vote_count': 'total_votes'
# }).reset_index()

# print(director_stats.sort_values(by='movie_count', ascending=False).head())
# print(actor_stats.sort_values(by='movie_count', ascending=False).head())


In [15]:
# director_stats.sort_values(by='total_votes', ascending=False).head()

In [16]:
final_filterd_tmdb_genres_drop = final_filterd_tmdb.dropna(subset=['genres'])

In [17]:
final_filterd_tmdb_director_drop = final_filterd_tmdb_genres_drop.dropna(subset=['director'])

In [18]:
threshold = final_filterd_tmdb_director_drop['vote_count'].quantile(0.99)
final_filterd_tmdb2 = final_filterd_tmdb_director_drop[final_filterd_tmdb_director_drop['vote_count'] >= threshold]

In [19]:
final_filterd_tmdb2.to_csv('final_filterd_tmdb2.csv',index=False )

# Recommendation System 'overview'

In [ ]:
final_filterd_tmdb2['overview'][:20]
final_filterd_tmdb2

In [21]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
final_filterd_tmdb2['overview'] = final_filterd_tmdb2['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(final_filterd_tmdb2['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

/var/folders/dr/lcj3g6gx3ps0px9m71zvyj680000gn/T/ipykernel_3259/2129588785.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filterd_tmdb2['overview'] = final_filterd_tmdb2['overview'].fillna('')


(4976, 19150)

In [22]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [23]:
indices = pd.Series(range(len(final_filterd_tmdb2)), index=final_filterd_tmdb2['title']).drop_duplicates()


In [24]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    if title not in indices:
        return []
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return final_filterd_tmdb2['title'].iloc[movie_indices]

In [25]:
final_filterd_tmdb2['title']

2488                                     Love Hard
4230                                         Troll
6327             Operation Fortune: Ruse de Guerre
7462                    Borat Subsequent Moviefilm
9924                                 After We Fell
                            ...                   
904548    The SpongeBob Movie: Sponge Out of Water
904561                    The Hundred-Foot Journey
904568                              McFarland, USA
904570                            The Longest Ride
904636                            The Book of Life
Name: title, Length: 4976, dtype: object

In [26]:
get_recommendations('Jurassic World')

741919    Jurassic World: Fallen Kingdom
775623     The Lost World: Jurassic Park
775622                     Jurassic Park
781127       National Lampoon's Vacation
776297                        Free Willy
416794                         Hell Fest
474265           Return to Jurassic Park
477479           Jurassic World Dominion
904386                       The Nut Job
695536                          Vacation
Name: title, dtype: object

# Recommendation System2

In [27]:
final_filterd_tmdb2['casts'].info()

<class 'pandas.core.series.Series'>
Index: 4976 entries, 2488 to 904636
Series name: casts
Non-Null Count  Dtype 
--------------  ----- 
4973 non-null   object
dtypes: object(1)
memory usage: 77.8+ KB


In [28]:
final_filterd_tmdb2.columns

Index(['id', 'title', 'budget', 'revenue', 'genres', 'imdb_id', 'overview',
       'tagline', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'vote_average',
       'vote_count', 'casts', 'director', 'director_of_photography',
       'producer', 'writer', 'music_composer'],
      dtype='object')

In [29]:

movie_data_selected = final_filterd_tmdb2[['id','title', 'genres', 'casts', 'director', 'writer', 'tagline','production_companies','director_of_photography', 'producer', 'music_composer']].fillna('')
missing_values = movie_data_selected.isnull().sum()
preprocessed_data = movie_data_selected.head()

missing_values, preprocessed_data


(id                         0
 title                      0
 genres                     0
 casts                      0
 director                   0
 writer                     0
 tagline                    0
 production_companies       0
 director_of_photography    0
 producer                   0
 music_composer             0
 dtype: int64,
             id                              title  \
 2488  734265.0                          Love Hard   
 4230  736526.0                              Troll   
 6327  739405.0  Operation Fortune: Ruse de Guerre   
 7462  740985.0         Borat Subsequent Moviefilm   
 9924  744275.0                      After We Fell   
 
                                    genres  \
 2488                      Comedy, Romance   
 4230           Fantasy, Action, Adventure   
 6327  Action, Thriller, Comedy, Adventure   
 7462                               Comedy   
 9924                       Romance, Drama   
 
                                                   

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
movie_data_selected['combined_features'] = movie_data_selected['genres'] + ' ' + movie_data_selected['casts'] + ' ' + movie_data_selected['director'] + ' ' + movie_data_selected['writer']+ ' ' + movie_data_selected['tagline']+ ' ' + movie_data_selected['production_companies']+ ' ' + movie_data_selected['director_of_photography']+ ' ' + movie_data_selected['producer']+ ' ' + movie_data_selected['music_composer']
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movie_data_selected['combined_features'])
tfidf_matrix.shape

(4976, 33499)

In [31]:
movie_data_selected2 = movie_data_selected.reset_index()

In [32]:
movie_data_selected

,id,title,genres,casts,director,writer,tagline,production_companies,director_of_photography,producer,music_composer,combined_features
2488,734265.0,Love Hard,"Comedy, Romance","['Nina Dobrev', 'Jimmy O. Yang', 'Darren Barne...",['Hernán Jiménez'],"['Daniel Mackey', 'Rebecca Ewing']",A romantic comedy about the lies we tell for l...,Wonderland Sound and Vision,['Shane Hurlbut'],"['McG', 'Mary Viola']",['Mark Orton'],"Comedy, Romance ['Nina Dobrev', 'Jimmy O. Yang..."
4230,736526.0,Troll,"Fantasy, Action, Adventure","['Ine Marie Wilmann', 'Kim S. Falck-Jørgensen'...",['Roar Uthaug'],"['Roar Uthaug', 'Espen Aukan']",Mountains will move.,Motion Blur,['Jallo Faber'],"['Espen Horn', 'Kristian Strand Sinkerud']",['Johannes Ringen'],"Fantasy, Action, Adventure ['Ine Marie Wilmann..."
6327,739405.0,Operation Fortune: Ruse de Guerre,"Action, Thriller, Comedy, Adventure","['Jason Statham', 'Aubrey Plaza', 'Josh Hartne...",['Guy Ritchie'],"['Guy Ritchie', 'Marn Davies', 'Ivan Atkinson']","In this operation, everyone has a part to play.","Miramax, STXfilms, Toff Guy Films, AZ Celtic F...",['Alan Stewart'],"['Guy Ritchie', 'Jason Statham', 'Steven Chasm...",['Chris Benstead'],"Action, Thriller, Comedy, Adventure ['Jason St..."
7462,740985.0,Borat Subsequent Moviefilm,Comedy,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",['Jason Woliner'],"['Sacha Baron Cohen', 'Anthony Hines', 'Peter ...",Delivery of prodigious bribe to American regim...,"Four by Two, Oak Springs Productions",['Luke Geissbuhler'],"['Sacha Baron Cohen', 'Anthony Hines', 'Monica...",['Erran Baron Cohen'],"Comedy ['Sacha Baron Cohen', 'Maria Bakalova',..."
9924,744275.0,After We Fell,"Romance, Drama","['Josephine Langford', 'Hero Fiennes Tiffin', ...",['Castille Landon'],"['Sharon Soboil', 'Anna Todd']",,"Voltage Pictures, Wattpad, Vertical Entertainm...",['Joshua Reis'],"['Jennifer Gibgot', 'Mark Canton', 'Courtney S...",['George Kallis'],"Romance, Drama ['Josephine Langford', 'Hero Fi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
904548,228165.0,The SpongeBob Movie: Sponge Out of Water,"Family, Adventure, Comedy, Fantasy, Animation","['Tom Kenny', 'Bill Fagerbakke', 'Rodger Bumpa...","['Mike Mitchell', 'Paul Tibbitt']","['Glenn Berger', 'Jonathan Aibel', 'Stephen Hi...",Making waves in our world.,"Paramount Animation, Paramount, Nickelodeon Mo...",['Phil Meheux'],"['Mary Parent', 'Nan Morales', 'Craig Sost']",['John Debney'],"Family, Adventure, Comedy, Fantasy, Animation ..."
904561,228194.0,The Hundred-Foot Journey,"Drama, Comedy","['Helen Mirren', 'Manish Dayal', 'Om Puri', 'C...",['Lasse Hallström'],"['Steven Knight', 'Richard C. Morais']",Life's greatest journey begins with the first ...,"Amblin Entertainment, Harpo Films, Participant...",['Linus Sandgren'],"['Steven Spielberg', 'Oprah Winfrey']",['A.R. Rahman'],"Drama, Comedy ['Helen Mirren', 'Manish Dayal',..."
904568,228203.0,"McFarland, USA","Drama, Family","['Kevin Costner', 'Ramiro Rodriguez', 'Maria B...",['Niki Caro'],"['Grant Thompson', 'Christopher Cleveland', 'B...",Champions can come from anywhere.,"Mayhem Pictures, Walt Disney Pictures",['Adam Arkapaw'],"['Mark Ciardi', 'Gordon Gray']",['Antonio Pinto'],"Drama, Family ['Kevin Costner', 'Ramiro Rodrig..."
904570,228205.0,The Longest Ride,"Drama, Romance","['Scott Eastwood', 'Britt Robertson', 'Lolita ...",['George Tillman Jr.'],"['Craig Bolotin', 'Nicholas Sparks']",Two couples. Two love stories. One epic tale.,"Fox 2000 Pictures, Temple Hill Entertainment",['David Tattersall'],"['Wyck Godfrey', 'Marty Bowen', 'Nicholas Spar...",['Mark Isham'],"Drama, Romance ['Scott Eastwood', 'Britt Rober..."


In [57]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = movie_data_selected2.index[movie_data_selected2['title'] == title].tolist()
    if not idx:
        return "Movie not found."
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return movie_data_selected2['title'].iloc[movie_indices].tolist()

get_recommendations("Kung Fu Panda")


['Kung Fu Panda 2',
 'Kung Fu Panda 3',
 'Trolls',
 '3-Iron',
 'The Croods',
 'Monsters vs Aliens',
 'Spring, Summer, Fall, Winter... and Spring',
 'Sinbad: Legend of the Seven Seas',
 'The SpongeBob Movie: Sponge on the Run',
 'The Interview']